In [3]:
#import libraries
import os
import numpy as np
import pandas as pd

#image processing
from PIL import Image, ImageOps

In [4]:
#load in the meta data as pandas data frame
#create one frame to store data as rgb, another to store it as grayscale
meta_train_rgb = pd.read_csv('petfinder-pawpularity-score/train.csv')
meta_train_grayscale = pd.read_csv('petfinder-pawpularity-score/train.csv')

In [12]:
#create a dummy vector of 0's so that we can vstack with new gray and rgb image vectors
gray_image_array_all = np.zeros(8100)
rgb_image_array_all = np.zeros(7500)

#loop over each for in the pandas data frame to grab the image id
for i in range(len(meta_train_grayscale)):
    #load image for row of meta data
    image = Image.open('petfinder-pawpularity-score/train/'+meta_train_grayscale['Id'][i]+'.jpg')
    #convert image to grayscale, resize it, convert to array, and flatten array
    gray_image = ImageOps.grayscale(image)
    resized_gray_image = gray_image.resize((90, 90))
    gray_image_array = np.array(resized_gray_image)
    gray_image_flat = gray_image_array.flatten()
    #stack the current grayscale image with the dummy vector created above
    gray_image_array_all = np.vstack((gray_image_array_all, gray_image_flat))
    
    #now we will repeat for the RGB images
    #resize image, convert to array, and flatten array
    resized_rgb_image = image.resize((50, 50))
    rgb_image_array = np.array(resized_rgb_image)
    rgb_image_flat = rgb_image_array.flatten()
    #stack the current grayscale image with the dummy vector created above
    rgb_image_array_all = np.vstack((rgb_image_array_all, rgb_image_flat))



In [20]:
#lets export the arrays containing image vectors because the above for loop took about an hour to run
np.savetxt("petfinder-pawpularity-score/pre_processed_data/pre_processed_images_grayscale.csv", gray_image_array_all, delimiter=",")
np.savetxt("petfinder-pawpularity-score/pre_processed_data/pre_processed_images_RGB.csv", rgb_image_array_all, delimiter=",")

In [32]:
#remove the first row of zeros the grayscale and RGB arrays
gray_image_array_all_ = gray_image_array_all[1:]
rgb_image_array_all_ = rgb_image_array_all[1:]

In [13]:
#next, we need to create column id's so we can convert the np arrays to dataframes to merge with meta data
gray_column_ids = []
for i in range(8100):
    i_str = str(i+1)
    gray_column_ids.append('p'+i_str)

rgb_column_ids = []
for i in range(7500):
    i_str = str(i+1)
    rgb_column_ids.append('p'+i_str)

In [33]:
#lets create the data frames
image_train_rgb = pd.DataFrame(rgb_image_array_all_, columns=rgb_column_ids)
image_train_gray = pd.DataFrame(gray_image_array_all_, columns=gray_column_ids)

In [34]:
#lastly, concatenate the meta data and the image data into one matrix
train_data_rgb = pd.concat([meta_train_rgb, image_train_rgb], axis=1)
train_data_gray = pd.concat([meta_train_grayscale, image_train_gray], axis=1)

In [35]:
#and for ease of splitting labels and data, lets move the pawpularity score to the beginning of the data frame
rgb_pawpularity = train_data_rgb.pop('Pawpularity')
train_data_rgb.insert(0, 'Pawpularity', rgb_pawpularity)
gray_pawpularity = train_data_gray.pop('Pawpularity')
train_data_gray.insert(0, 'Pawpularity', rgb_pawpularity)


In [40]:
#export both data frames to csv files
train_data_rgb.to_csv('meta_image_rgb_data.csv')

In [39]:
train_data_gray.to_csv('petfinder-pawpularity-score/meta_image_gray_data.csv')